In [1]:
import pandas as pd

# Data preparation

## Data preparation (from row_data)

In [2]:
Test_df = pd.read_csv('../raw_data/Corona_NLP_test.csv')
Train_df = pd.read_csv('../raw_data/Corona_NLP_train.csv',encoding_errors='ignore')

In [3]:
Test_df.drop(columns=['UserName','ScreenName','Location','TweetAt'],inplace=True)
Train_df.drop(columns=['UserName','ScreenName','Location','TweetAt'],inplace=True)

In [4]:
Test_df.to_csv('../Demo_Project_NLP_sentiment_analysis_benbhk/data/X_test.csv')
Train_df.to_csv('../Demo_Project_NLP_sentiment_analysis_benbhk/data/X_train.csv')

## Data loading (from data)

In [5]:
Test_df = pd.read_csv('../Demo_Project_NLP_sentiment_analysis_benbhk/data/X_test.csv')
Train_df = pd.read_csv('../Demo_Project_NLP_sentiment_analysis_benbhk/data/X_train.csv')
Test_df.dropna(inplace=True)
Train_df.dropna(inplace=True)

In [6]:
Test_df.drop(columns=['Unnamed: 0'],inplace=True)
Train_df.drop(columns=['Unnamed: 0'],inplace=True)

## Data Processing

In [7]:
Test_df['Sentiment_num'] = -1
Test_df.loc[Test_df['Sentiment'] =='Extremely Negative','Sentiment_num'] = 0
Test_df.loc[Test_df['Sentiment'] =='Negative','Sentiment_num'] = 1
Test_df.loc[Test_df['Sentiment'] =='Neutral','Sentiment_num'] = 2
Test_df.loc[Test_df['Sentiment'] =='Positive','Sentiment_num'] = 3
Test_df.loc[Test_df['Sentiment'] =='Extremely Positive','Sentiment_num'] = 4

Train_df.loc[Train_df['Sentiment'] =='Extremely Negative','Sentiment_num'] = 0
Train_df.loc[Train_df['Sentiment'] =='Negative','Sentiment_num'] = 1
Train_df.loc[Train_df['Sentiment'] =='Neutral','Sentiment_num'] = 2
Train_df.loc[Train_df['Sentiment'] =='Positive','Sentiment_num'] = 3
Train_df.loc[Train_df['Sentiment'] =='Extremely Positive','Sentiment_num'] = 4

# Test_df[Test_df['Sentiment']=='Extremely Negative']['Sentiment_num'] = 0

In [8]:
X_test = Test_df['OriginalTweet'].to_numpy()
X_train = Train_df['OriginalTweet'].to_numpy()
y_test = Test_df['Sentiment_num'].to_numpy(dtype=int)
y_train = Train_df['Sentiment_num'].to_numpy(dtype=int)

In [9]:
print(X_test.shape,y_test.shape)

(3798,) (3798,)


In [10]:
print(X_train.shape,y_train.shape)

(41155,) (41155,)


In [11]:
for i in range(len(X_train)):
    X_train[i] = str.encode(X_train[i])
for i in range(len(X_test)):
    X_test[i] = str.encode(X_test[i])

In [12]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in X_train]
X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in X_test]

2022-04-21 17:57:52.090506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 17:57:52.090568: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

## Model création

In [23]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Masking(mask_value=0, input_shape=(200,100)))
model.add(layers.LSTM(units=20, activation="tanh"))
model.add(layers.Dense(10,activation="relu"))
model.add(layers.Dense(5,activation="softmax"))

2022-04-21 17:56:10.013123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-21 17:56:10.013202: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-21 17:56:10.013235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BenLaptop-V8N6C5JR): /proc/driver/nvidia/version does not exist
2022-04-21 17:56:10.013759: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

es = EarlyStopping(patience=10,restore_best_weights=True)

history = model.fit(X_train_pad, y_train,
                    validation_split=0.2,
                    epochs=20, 
                    callbacks=[es],
                    batch_size=32, 
                    verbose=1)

NameError: name 'X_train_pad' is not defined

In [14]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [15]:
word2vec_transfer = api.load('glove-wiki-gigaword-50')

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
# from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Masking(mask_value=0, input_shape=(200,50)))
model.add(layers.LSTM(units=20, activation="tanh"))
model.add(layers.Dense(10,activation="relu"))
model.add(layers.Dense(1,activation="softmax"))

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

es = EarlyStopping(patience=10,restore_best_weights=True)

history = model.fit(X_train_pad_2, y_train,
                    validation_split=0.2,
                    epochs=500, 
                    callbacks=[es],
                    batch_size=32, 
                    verbose=1)

NameError: name 'model' is not defined